In [1]:
import os
import numpy as np
import librosa
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
import joblib  # Untuk menyimpan model SVM

In [2]:
# Direktori dataset
INPUT_DIR = 'dataset'

# Direktori tempat menyimpan model
MODEL_DIR = 'model'
# Buat direktori jika belum ada
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [3]:
# Fungsi untuk load dan ekstrak Mel-spectrogram
def load_and_extract_spectrogram(file_path, n_mels=128, n_fft=2048, hop_length=512, max_time_steps=128):
    y, sr = librosa.load(file_path, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    if mel_spec_db.shape[1] < max_time_steps:
        pad_width = max_time_steps - mel_spec_db.shape[1]
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_spec_db = mel_spec_db[:, :max_time_steps]
    return mel_spec_db.flatten()  # Flatten untuk SVM


In [4]:
# Loop melalui setiap folder (kelas)
subdirectories = [subdir for subdir in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, subdir))]


In [5]:
for subdir in subdirectories:
    print(f"Processing class: {subdir}")
    subdir_path = os.path.join(INPUT_DIR, subdir)
    
    # Load dataset untuk kelas ini
    X = []
    wav_files = [file for file in os.listdir(subdir_path) if file.endswith('.wav')]
    
    for wav_file in wav_files:
        file_path = os.path.join(subdir_path, wav_file)
        spectrogram = load_and_extract_spectrogram(file_path)
        X.append(spectrogram)  

    # Konversi ke numpy array
    X = np.array(X)

    # Split dataset menjadi train dan test
    X_train, X_test = train_test_split(X, test_size=0.15, random_state=42)

    # Inisialisasi model One-Class SVM
    model = OneClassSVM(gamma=0.000000001).fit(X_train)  # Melatih hanya pada data benar

    # Simpan model ke dalam direktori 'model'
    model_filename = os.path.join(MODEL_DIR, f'one_class_svm_{subdir}.joblib')
    joblib.dump(model, model_filename)
    print(f"Model saved for class {subdir} at {model_filename}")

    # Evaluasi model pada data pengujian
    y_pred_test = model.predict(X_test)
    print(y_pred_test)
    
    # Hasil prediksi: 1 = inlier (benar), -1 = outlier (salah)
    accuracy = np.mean(y_pred_test == 1) * 100  # Persentase yang dianggap benar
    print(f"Test accuracy (detection of correct class) for {subdir}: {accuracy:.2f}%")

Processing class: 01. alif_fathah
Model saved for class 01. alif_fathah at model\one_class_svm_01. alif_fathah.joblib
[-1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1]
Test accuracy (detection of correct class) for 01. alif_fathah: 43.75%
Processing class: 02. alif_kasroh
Model saved for class 02. alif_kasroh at model\one_class_svm_02. alif_kasroh.joblib
[-1  1 -1 -1  1  1 -1 -1 -1  1  1 -1  1 -1 -1  1]
Test accuracy (detection of correct class) for 02. alif_kasroh: 43.75%
Processing class: 03. alif_dommah
Model saved for class 03. alif_dommah at model\one_class_svm_03. alif_dommah.joblib
[ 1  1 -1 -1  1  1 -1 -1  1  1 -1 -1  1  1 -1 -1]
Test accuracy (detection of correct class) for 03. alif_dommah: 50.00%
Processing class: 04. ba_fathah
Model saved for class 04. ba_fathah at model\one_class_svm_04. ba_fathah.joblib
[ 1  1  1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1 -1  1]
Test accuracy (detection of correct class) for 04. ba_fathah: 50.00%
Processing class: 05. ba_kasroh
Model saved for clas

In [7]:
# Fungsi untuk memuat model dan melakukan prediksi
def predict_class(model_path, audio_path):
    # Muat model
    model = joblib.load(model_path)
    print(f"Model loaded from {model_path}")
    
    # Proses file audio
    audio_features = load_and_extract_spectrogram(audio_path)
    
    # Prediksi
    prediction = model.predict([audio_features])

    # Interpretasi hasil prediksi
    if prediction[0] == 1:
        print("Prediksi: Data cocok dengan kelas ini (benar).")
    else:
        print("Prediksi: Data tidak cocok dengan kelas ini (salah).")


model_path = 'model/one_class_svm_01. alif_fathah.joblib'
audio_path = 'a.wav'

predict_class(model_path, 'ta.wav')
predict_class('model/one_class_svm_02. alif_kasroh.joblib', 'i.wav')
predict_class('model/one_class_svm_03. alif_dommah.joblib', 'u.wav')
predict_class('model/one_class_svm_07. ta_fathah.joblib', 'ta.wav')

Model loaded from model/one_class_svm_01. alif_fathah.joblib
Prediksi: Data tidak cocok dengan kelas ini (salah).
Model loaded from model/one_class_svm_02. alif_kasroh.joblib
Prediksi: Data cocok dengan kelas ini (benar).
Model loaded from model/one_class_svm_03. alif_dommah.joblib
Prediksi: Data cocok dengan kelas ini (benar).
Model loaded from model/one_class_svm_07. ta_fathah.joblib
Prediksi: Data tidak cocok dengan kelas ini (salah).
